In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import math 

import sys
sys.path.append('../')
import utils
import wiki_utils
%matplotlib inline

In [2]:
corpus = wiki_utils.Texts('./wikitext/')

KeyboardInterrupt: 

In [ ]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

In [ ]:
eval_batch_size = 128
train_loader = wiki_utils.TextLoader(corpus.train, batch_size=batch_size)
val_loader = wiki_utils.TextLoader(corpus.valid, batch_size=eval_batch_size)
test_loader = wiki_utils.TextLoader(corpus.test, batch_size=eval_batch_size)
print ('train=', train_loader)
print ('val=', val_loader)
print ('test=', test_loader)


In [2]:
train_set = wiki_utils.TextDataset('./wikitext/', train=True)

./wikitext/train.txt


In [9]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [10]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    for i, (data, targets) in enumerate(data_loader):
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / len(data_loader)

In [11]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    for batch, (data, targets) in enumerate(train_loader):
    #for batch, i in enumerate(range(0, train_data.size(0) - 1, sequence_length)):
        #data, targets = get_batch(train_data, i)
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader) // sequence_length, lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [12]:
ntokens = len(corpus.dictionary)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3)
criterion = nn.CrossEntropyLoss()

In [13]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = corpus.dictionary.idx2symbol[s_idx]
        out.append(s)
    return ''.join(out)

In [ ]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


sample:
 ルоDHDهưCcØ\Sá−ح²حل₹½殻ắоγาė7ュიpzO„ôตСH〈ảF
्Đ攻/Í;4̃L 

| epoch   1 |   100/ 2807 batches | lr 4.00 | loss  3.60 | ppl    36.47
| epoch   1 |   200/ 2807 batches | lr 4.00 | loss  3.29 | ppl    26.73
| epoch   1 |   300/ 2807 batches | lr 4.00 | loss  3.25 | ppl    25.81
| epoch   1 |   400/ 2807 batches | lr 4.00 | loss  3.23 | ppl    25.17
| epoch   1 |   500/ 2807 batches | lr 4.00 | loss  3.22 | ppl    25.10
| epoch   1 |   600/ 2807 batches | lr 4.00 | loss  3.13 | ppl    22.87
| epoch   1 |   700/ 2807 batches | lr 4.00 | loss  3.00 | ppl    20.13
| epoch   1 |   800/ 2807 batches | lr 4.00 | loss  2.90 | ppl    18.22
| epoch   1 |   900/ 2807 batches | lr 4.00 | loss  2.81 | ppl    16.69
| epoch   1 |  1000/ 2807 batches | lr 4.00 | loss  2.73 | ppl    15.35
| epoch   1 |  1100/ 2807 batches | lr 4.00 | loss  2.63 | ppl    13.86
| epoch   1 |  1200/ 2807 batches | lr 4.00 | loss  2.56 | ppl    12.90
| epoch   1 |  1300/ 2807 batches | lr 4.00 | loss  2.50 | ppl    12.19
| 

| epoch   4 |  1800/ 2807 batches | lr 4.00 | loss  1.75 | ppl     5.73


In [ ]:
t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w') as outf:
    outf.write(t075)
with open('./generated1.txt', 'w') as outf:
    outf.write(t1)
with open('./generated15.txt', 'w') as outf:
    outf.write(t15)